In [1]:
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, PredefinedSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics

In [2]:
from feature import time_series_features, fractal_features, freq_band_features, hjorth_features, entropy_features
from dataset1 import load_dataset, load_labels, split_data, format_labels, load_channels
import variables as v
import genetic_algrthm as ga

In [3]:
data_type = "raw" #"ica_filtered"
test_type = "Arithmetic"


In [4]:
dataset_ = load_dataset(data_type=data_type, test_type=test_type)
dataset = split_data(dataset_, v.SFREQ)
print(f"Berhasil memuat dataset dengan {dataset.shape[2]} channel.")
print(f"Bentuk data: {dataset.shape}")

channels_list = load_channels()

ga_parameters = {
    "num_generations": 10,
    "num_parents_mating": 5,
    "num_people_in_pop": 15,
    "num_channels_to_select": 8
}

label_ = load_labels()
# num_epochs = 12
label = format_labels(label_, test_type=test_type, epochs= dataset.shape[1])

print("\n--- Menjalankan Algoritma Genetika untuk Seleksi Channel ---")
selected_channels = ga.run_channel_selection(
    full_dataset = dataset ,
    all_channels_list = channels_list,
    labels_for_ga = label,
    ga_params = ga_parameters
)

Berhasil memuat dataset dengan 32 channel.
Bentuk data: (120, 25, 32, 128)

--- Menjalankan Algoritma Genetika untuk Seleksi Channel ---
  -> Generasi 1/10...
     -> Ditemukan solusi terbaik baru dengan fitness 2.40
     -> Channel: ['O2' 'C3' 'CP5' 'Pz' 'CP6' 'FC5' 'P7' 'PO9']
  -> Generasi 2/10...
     -> Ditemukan solusi terbaik baru dengan fitness 2.42
     -> Channel: ['O2' 'C3' 'CP5' 'Pz' 'F4' 'FC1' 'T7' 'FC6']
  -> Generasi 3/10...
  -> Generasi 4/10...
  -> Generasi 5/10...
  -> Generasi 6/10...
  -> Generasi 7/10...
  -> Generasi 8/10...
  -> Generasi 9/10...
  -> Generasi 10/10...

--- Seleksi Channel Selesai ---
Channel terbaik yang ditemukan: ['O2', 'C3', 'CP5', 'Pz', 'F4', 'FC1', 'T7', 'FC6']


In [12]:
Data = ga.get_subset(
    data = dataset,
    all_genes = channels_list,
    subset_genes = selected_channels
)

print(f"\nChannel yang terpilih: {selected_channels}")
print(f"Bentuk data setelah seleksi channel: {Data.shape}")


Channel yang terpilih: ['O2', 'C3', 'CP5', 'Pz', 'F4', 'FC1', 'T7', 'FC6']
Bentuk data setelah seleksi channel: (120, 25, 8, 128)


In [9]:
# import mne
# import matplotlib.pyplot as plt

# # METADATA UNTUK MNE (MNE Info Object)
# # channel, frekuensi sampling, dan tipe channel.

# sfreq = v.SFREQ  # Frekuensi sampling (128 Hz)
# ch_names_all = channels_list  
# ch_types_all = ['eeg'] * len(ch_names_all) 

# # objek info untuk 32 channel (sebelum seleksi)
# info_all_channels = mne.create_info(
#     ch_names=ch_names_all, sfreq=sfreq, ch_types=ch_types_all)

# # objek info HANYA untuk 8 channel yang sudah terseleksi
# ch_names_selected = selected_channels 
# ch_types_selected = ['eeg'] * len(ch_names_selected)
# info_selected_channels = mne.create_info(
#     ch_names=ch_names_selected, sfreq=sfreq, ch_types=ch_types_selected)

# print("Metadata MNE berhasil dibuat.")

In [10]:
# # Reshape data dari (trials, channels, samples) menjadi (channels, samples)
# # dengan menggabungkan semua trial
# n_trials, n_channels, n_samples = dataset_.shape
# continuous_data = dataset_.reshape(n_channels, n_trials * n_samples)

# # objek MNE Raw dari NumPy array
# raw_mne = mne.io.RawArray(continuous_data, info_all_channels)

# # 1. Plot Time Series
# print("Menampilkan plot sinyal EEG dari waktu ke waktu...")
# raw_mne.plot(duration=30, n_channels=32, scalings='auto', block=True)

# # 2. Plot Power Spectral Density (PSD) / Spektrum
# print("Menampilkan plot spektrum frekuensi...")
# raw_mne.plot_psd(fmax=50)  # Tampilkan frekuensi hingga 50 Hz

In [13]:
from feature import differential_entropy_features

# features = time_series_features(Data)
# freq_bands = np.array([1, 4, 8, 12, 30, 50])
# features = freq_band_features(Data, freq_bands)
freq_bands = np.array([4, 8, 14, 31, 45])
features = differential_entropy_features(Data, freq_bands)
# features = hjorth_features(Data)
# features = entropy_features(Data)
# features = fractal_features(Data)

ValueError: operands could not be broadcast together with shapes (120,24,8,65) (120,1,25,8) 

In [ ]:
data = features

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

x, x_test, y, y_test = train_test_split(
    data, label, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(
    x, y, test_size=0.25, random_state=1)
scaler = MinMaxScaler()
scaler.fit(x_train)
x = scaler.transform(x)
x_train = scaler.transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

param_grid = {
    'leaf_size': range(1, 50),
    'n_neighbors': range(1, 10),
    'p': [1, 2]
}
split_index = [-1 if x in range(len(x_train)) else 0 for x in range(len(x))]
ps = PredefinedSplit(test_fold=split_index)
knn_clf = GridSearchCV(KNeighborsClassifier(), param_grid, cv=ps, refit=True)
knn_clf.fit(x, y)

,estimator,KNeighborsClassifier()
,param_grid,"{'leaf_size': range(1, 50), 'n_neighbors': range(1, 10), 'p': [1, 2]}"
,scoring,None
,n_jobs,None
,refit,True
,cv,"PredefinedSpl......, 0, 0]))"
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_neighbors,3


In [ ]:

y_pred = knn_clf.predict(x_test)
y_true = y_test

In [ ]:

print(metrics.classification_report(y_true, y_pred))
print(metrics.confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

       False       0.66      0.72      0.69       311
        True       0.67      0.61      0.64       289

    accuracy                           0.67       600
   macro avg       0.67      0.66      0.66       600
weighted avg       0.67      0.67      0.67       600

[[225  86]
 [114 175]]
